In [ ]:
from google.colab import drive
drive.mount('/content/gdrive/')

Mounted at /content/gdrive/


In [ ]:
import numpy as np
import pandas as pd

traffic_df = pd.read_csv('/content/gdrive/MyDrive/CIC-Darknet/darknet_log.csv')
traffic_df.shape

(141481, 80)

### Drop Columns

In [ ]:
drop_columns = [2, 5, 8, 9, 10, 12, 13, 14, 15, 18, 19, 23, 24, 25, 28, 32, 33, 34, 35, 37, 40, 41, 42, 44, 45, 46, 47, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 63, 64, 65, 66, 69, 70, 71, 72, 73, 74, 76]
traffic_df.drop(traffic_df.columns[drop_columns], axis=1 ,inplace=True)
traffic_df.head()

,Src Port,Dst Port,Flow Duration,Total Fwd Packet,Total Length of Fwd Packet,Total Length of Bwd Packet,Fwd Packet Length Std,Flow Bytes/s,Flow Packets/s,Flow IAT Max,...,Packet Length Std,PSH Flag Count,Bwd Packet/Bulk Avg,Bwd Bulk Rate Avg,FWD Init Win Bytes,Bwd Init Win Bytes,Idle Mean,Idle Max,Idle Min,target
0,57158,443,229,1,0,0,0.000,0.000000,9.075050,229,...,0.00,0,0,0,1892,1047,0.000000,0.000000,0.000000,0
1,57159,443,407,1,0,0,0.000,0.000000,8.500048,407,...,0.00,0,0,0,1987,1047,0.000000,0.000000,0.000000,0
2,57160,443,431,1,0,0,0.000,0.000000,8.442765,431,...,0.00,0,0,0,2049,1047,0.000000,0.000000,0.000000,0
3,49134,443,359,1,0,0,0.000,0.000000,8.625515,359,...,0.00,0,0,0,2008,1047,0.000000,0.000000,0.000000,0
4,34697,19305,10778451,591,64530,6659,3.148,8.795696,4.531983,78158,...,4.06,659,659,6605,1382,2320,34.901863,34.901863,34.901863,0


### Data Split

In [ ]:
from sklearn.model_selection import train_test_split

y_traffic_df = traffic_df['target']
X_traffic_df = traffic_df.drop('target', axis=1)

X_train, X_test, y_train, y_test = train_test_split(X_traffic_df, y_traffic_df, stratify=y_traffic_df, test_size=0.4, random_state=11)
print(X_train.shape, y_train.shape)
print(X_test.shape, y_test.shape)

(84888, 29) (84888,)
(56593, 29) (56593,)


In [ ]:
X_vali, X_test, y_vali, y_test = train_test_split(X_test, y_test, stratify=y_test, test_size=0.5, random_state=11)
print(X_vali.shape, y_vali.shape)
print(X_test.shape, y_test.shape)

(28296, 29) (28296,)
(28297, 29) (28297,)


In [ ]:
label_name = ['benign_Audio-Streaming', 'benign_Browsing', 'benign_Chat', 'benign_Email', 'benign_File-Transfer', 'benign_P2P', 'benign_VOIP', 'benign_Video-Streaming', 'darknet_Audio-Streaming', 'darknet_Browsing', 'darknet_Chat', 'darknet_Email', 'darknet_File-Transfer', 'darknet_P2P', 'darknet_VOIP', 'darknet_Video-Streaming']
len(label_name)

16

### Model Train / Validation

In [ ]:
from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report
from lightgbm import LGBMClassifier
import warnings
warnings.filterwarnings('ignore')

lgbm_wrapper = LGBMClassifier(random_state=11, max_depth=10, learning_rate=0.046690609202960004, n_estimators=1000, colsample_bytree=0.5449634467189419, colsample_bylevel=0.6170260924043028, colsample_bynode=0.5982217573905297, reg_lambda=0.02395878521182272, reg_alpha=0.6772588890679594, subsample=0.85, min_child_weight=2, gamma=0.32949337881908053, boost_from_average=False)
lgbm_wrapper.fit(X_train, y_train)
lgbm_pred = lgbm_wrapper.predict(X_vali)

print('{0} 정확도:{1:.4f}'.format(lgbm_wrapper.__class__.__name__, accuracy_score(y_vali, lgbm_pred)))
print(confusion_matrix(y_vali, lgbm_pred), '\n')
print(classification_report(y_vali, lgbm_pred, target_names=label_name), '\n')

LGBMClassifier 정확도:0.9075
[[ 631   32    4    1   14   14    2  220   30    0    2    0    1    0
     0    3]
 [   3 6394    2    0   65    2    0   41    0    0    0    0    1    0
     0    1]
 [   5   18  987  203    7    4  106   15    5    0    2    0    1    0
    33    0]
 [   0    3  397  603    3    2   71    1    4    0    0    2    9    0
    17    0]
 [  14  200   16    4 1413    9    6   42    2    0    0    0    1    0
     2    3]
 [   0    0    0    0    2 9657    0    0    1    0    0    0    0    0
     0    0]
 [   0    0  135   64    0    0  211    1    1    0    0    0    1    0
     7    0]
 [ 240  133    3    1   48    3    0 1217   28    0    3    0    2    0
     0    2]
 [  15    1    2    3    0    0    0   23 2590    0    7    0    8    1
     0    7]
 [   0    0    0    2    0    0    0    0    2   48    0    0    0    0
     0    1]
 [   1    0    1    1    0    0    0   11    7    1  882    2    1    0
     0    1]
 [   0    2    0    3    0    0    0   

### Model Test

In [ ]:
lgbm_test = lgbm_wrapper.predict(X_test)

print('{0} 정확도:{1:.4f}'.format(lgbm_wrapper.__class__.__name__, accuracy_score(y_test, lgbm_test)))
print(confusion_matrix(y_test, lgbm_test), '\n')
print(classification_report(y_test, lgbm_test, target_names=label_name), '\n')

LGBMClassifier 정확도:0.9120
[[ 651   18    2    3   20   13    1  204   35    0    2    0    2    0
     0    2]
 [   1 6406    1    0   59    7    0   33    1    0    0    0    0    0
     0    1]
 [   1   19 1001  201    2    7  109   11    3    0    1    0    0    0
    32    0]
 [   0    1  375  631    9    0   68    1    0    0    0    8    6    0
    14    0]
 [  16  180   17    6 1450    6    8   22    2    0    0    0    0    0
     2    4]
 [   0    4    0    0    0 9655    0    0    1    0    0    0    0    0
     0    0]
 [   0    1  134   55    3    0  215    2    0    0    0    0    0    0
    10    0]
 [ 222  127    3    3   34    3    0 1246   34    0    2    1    0    0
     2    4]
 [   8    1    0    1    1    0    0   32 2578    4    9    0    6    1
     0   16]
 [   0    0    0    1    0    0    0    0    1   47    1    0    0    0
     2    0]
 [   0    0    1    1    0    0    0    5    5    0  887    1    4    0
     0    4]
 [   0    3    0    2    0    0    0   